In [1]:
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time

In [2]:
#Authentication token
os.environ['TOKEN'] = ''

In [3]:
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

# Utility

In [5]:
def create_url(keyword, start_date, end_date, max_results):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #endpoint

    #params
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}} 
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token):
    params['next_token'] = next_token   
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
def append_to_csv(json_response, fileName):

    counter = 0

    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    for tweet in json_response['data']:

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        res = [author_id, created_at, geo, tweet_id ,lang, like_count, quote_count, reply_count, retweet_count, source , text]
    
        csvWriter.writerow(res)
        counter += 1

    csvFile.close()

    print("# of Tweets added from this response: ", counter) 

# Looping query

In [ ]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
#Insert query to search over Twitter
keyword = "#dpcgov OR #protezionecivile lang:it"
start_list =    ['2021-01-01T00:00:00.000Z',
                '2021-02-01T00:00:00.000Z',
                '2021-03-01T00:00:00.000Z',
                '2021-04-01T00:00:00.000Z',
                '2021-05-01T00:00:00.000Z',
                '2021-06-01T00:00:00.000Z',
                '2021-07-01T00:00:00.000Z',
                '2021-08-01T00:00:00.000Z',
                '2021-09-01T00:00:00.000Z',
                '2021-10-01T00:00:00.000Z',
                '2021-11-01T00:00:00.000Z',
                '2021-12-01T00:00:00.000Z']
end_list =      ['2021-01-31T00:00:00.000Z',
                '2021-02-28T00:00:00.000Z',
                '2021-03-31T00:00:00.000Z',
                '2021-04-30T00:00:00.000Z',
                '2021-05-31T00:00:00.000Z',
                '2021-06-30T00:00:00.000Z',
                '2021-07-31T00:00:00.000Z',
                '2021-08-31T00:00:00.000Z',
                '2021-09-30T00:00:00.000Z',
                '2021-10-31T00:00:00.000Z',
                '2021-11-30T00:00:00.000Z',
                '2021-12-31T00:00:00.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("Hashtags.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'tweet_id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count', 'source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 50000 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "Hashtags.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "Hashtags.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)